##### Copyright 2023 Google LLC

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import


In [ ]:
!pip install -U -q google-generativeai

In [ ]:
# import necessary modules.
import google.generativeai as genai
import json
import base64
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown

## Set the API key

Add your API_KEY to the secrets manager in the left pannel "🔑".

In [ ]:
from google.colab import userdata

API_KEY=userdata.get('API_KEY')

In [ ]:
# Configure the client library by providing your API key.
genai.configure(api_key=API_KEY)

### Parse the arguments

In [ ]:
model = 'gemini-1.5-pro-latest' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6IkNSSUUgVU0gU0lTVEVNQSBQQVJBIEJVU0NBIEVNIERPQ1VNRU5UT1MgVVNBTkRPIEVNQkVERElOR1MgRSBBIEdFTUlOSSBBUEkifSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjoiIyMgU2lzdGVtYSBkZSBCdXNjYSBlbSBEb2N1bWVudG9zIGNvbSBFbWJlZGRpbmdzIGUgR2VtaW5pIEFQSVxuXG5Fc3RlIHNpc3RlbWEgdXRpbGl6YSBlbWJlZGRpbmdzIGdlcmFkb3MgcGVsYSBHZW1pbmkgQVBJIHBhcmEgYnVzY2FyIGRvY3VtZW50b3MgcmVsZXZhbnRlcyBhIHBhcnRpciBkZSB1bWEgY29uc3VsdGEgdGV4dHVhbC4gXG5cbioqQ29tcG9uZW50ZXM6KipcblxuMS4gKipHZXJhw6fDo28gZGUgRW1iZWRkaW5nczoqKlxuICAgICogVXRpbGl6YXJlbW9zIGEgR2VtaW5pIEFQSSBwYXJhIGdlcmFyIGVtYmVkZGluZ3MgZGUgYWx0YSBxdWFsaWRhZGUgcGFyYSBjYWRhIGRvY3VtZW50byBlIHBhcmEgYSBjb25zdWx0YSBkbyB1c3XDoXJpby4gXG4gICAgKiBPcyBlbWJlZGRpbmdzIGNhcHR1cmFtIG8gc2lnbmlmaWNhZG8gc2Vtw6JudGljbyBkbyB0ZXh0bywgcGVybWl0aW5kbyBjb21wYXJhw6fDtWVzIGRlIHNpbWlsYXJpZGFkZS5cblxuMi4gKipBcm1hemVuYW1lbnRvIGRlIEVtYmVkZGluZ3M6KipcbiAgICAqIE9zIGVtYmVkZGluZ3MgZG9zIGRvY3VtZW50b3Mgc2Vyw6NvIGFybWF6ZW5hZG9zIGVtIHVtIGJhbmNvIGRlIGRhZG9zLCBqdW50YW1lbnRlIGNvbSBzZXVzIG1ldGFkYWRvcyAodMOtdHVsbywgYXV0b3IsIGV0Yy4pLlxuICAgICogT3DDp8O1ZXM6XG4gICAgICAgICogKipDaHJvbWE6KiogQmFuY28gZGUgZGFkb3MgdmV0b3JpYWwgb3RpbWl6YWRvIHBhcmEgYnVzY2EgcG9yIHNpbWlsYXJpZGFkZS5cbiAgICAgICAgKiAqKkZhaXNzOioqIEJpYmxpb3RlY2EgcGFyYSBidXNjYSBlZmljaWVudGUgZW0gZXNwYcOnb3MgdmV0b3JpYWlzLlxuICAgICAgICAqICoqUGluZWNvbmU6KiogU2VydmnDp28gZGUgdmV0b3JlcyBuYSBudXZlbSwgZXNjYWzDoXZlbCBlIGRlIGbDoWNpbCBpbnRlZ3Jhw6fDo28uXG5cbjMuICoqQnVzY2E6KipcbiAgICAqIEEgY29uc3VsdGEgZG8gdXN1w6FyaW8gc2Vyw6EgY29udmVydGlkYSBlbSB1bSBlbWJlZGRpbmcgdXRpbGl6YW5kbyBhIEdlbWluaSBBUEkuXG4gICAgKiBPIGVtYmVkZGluZyBkYSBjb25zdWx0YSBzZXLDoSBjb21wYXJhZG8gY29tIG9zIGVtYmVkZGluZ3MgZG9zIGRvY3VtZW50b3Mgbm8gYmFuY28gZGUgZGFkb3MsIHV0aWxpemFuZG8gdW1hIG3DqXRyaWNhIGRlIHNpbWlsYXJpZGFkZSAoZXg6IGRpc3TDom5jaWEgY29zc2VubykuXG4gICAgKiBPcyBkb2N1bWVudG9zIG1haXMgc2ltaWxhcmVzIMOgIGNvbnN1bHRhIHNlcsOjbyByZXRvcm5hZG9zIGNvbW8gcmVzdWx0YWRvIGRhIGJ1c2NhLCByYW5xdWVhZG9zIHBvciByZWxldsOibmNpYS5cblxuKipGbHV4b2dyYW1hOioqXG5cbjEuICoqSW5kZXhhw6fDo286KipcbiAgICAqIFBhcmEgY2FkYSBkb2N1bWVudG86XG4gICAgICAgICogRXh0cmFpciB0ZXh0byBkbyBkb2N1bWVudG8uXG4gICAgICAgICogR2VyYXIgZW1iZWRkaW5nIGRvIHRleHRvIHV0aWxpemFuZG8gYSBHZW1pbmkgQVBJLlxuICAgICAgICAqIEFybWF6ZW5hciBvIGVtYmVkZGluZyBlIG9zIG1ldGFkYWRvcyBkbyBkb2N1bWVudG8gbm8gYmFuY28gZGUgZGFkb3MuXG5cbjIuICoqQnVzY2E6KipcbiAgICAqIFJlY2ViZXIgYSBjb25zdWx0YSB0ZXh0dWFsIGRvIHVzdcOhcmlvLlxuICAgICogR2VyYXIgZW1iZWRkaW5nIGRhIGNvbnN1bHRhIHV0aWxpemFuZG8gYSBHZW1pbmkgQVBJLlxuICAgICogQ29uc3VsdGFyIG8gYmFuY28gZGUgZGFkb3MsIGNvbXBhcmFuZG8gbyBlbWJlZGRpbmcgZGEgY29uc3VsdGEgY29tIG9zIGVtYmVkZGluZ3MgZG9zIGRvY3VtZW50b3MuXG4gICAgKiBSZWN1cGVyYXIgb3MgZG9jdW1lbnRvcyBtYWlzIHNpbWlsYXJlcyDDoCBjb25zdWx0YSwgcmFucXVlYWRvcyBwb3IgcmVsZXbDom5jaWEuXG4gICAgKiBBcHJlc2VudGFyIG9zIHJlc3VsdGFkb3MgZGEgYnVzY2EgYW8gdXN1w6FyaW8uXG5cbioqRXhlbXBsbyBkZSBjw7NkaWdvIFB5dGhvbiAodXRpbGl6YW5kbyBDaHJvbWEpOioqXG5cbmBgYHB5dGhvblxuZnJvbSBnb29nbGUuZ2VuZXJhdGl2ZWFpIGltcG9ydCBHZW1pbmlcbmZyb20gY2hyb21hZGIuY29uZmlnIGltcG9ydCBTZXR0aW5nc1xuZnJvbSBjaHJvbWFkYi51dGlscyBpbXBvcnQgZW1iZWRkaW5nX2Z1bmN0aW9uc1xuXG4jIENvbmZpZ3VyYXIgR2VtaW5pIEFQSVxuZ2VtaW5pID0gR2VtaW5pKClcbmdlbWluaS5zZXRfYXBpX2tleShcIlNVQV9BUElfS0VZXCIpXG5cbiMgQ29uZmlndXJhciBDaHJvbWFcbmNsaWVudCA9IGNocm9tYWRiLkNsaWVudChTZXR0aW5ncyhcbiAgICBjaHJvbWFfZGJfaW1wbD1cImR1Y2tkYitwYXJxdWV0XCIsXG4gICAgcGVyc2lzdF9kaXJlY3Rvcnk9XCIuL2Nocm9tYV9kYlwiXG4pKVxuXG4jIEZ1bsOnw6NvIHBhcmEgZ2VyYXIgZW1iZWRkaW5ncyBjb20gR2VtaW5pXG5kZWYgZ2VyYXJfZW1iZWRkaW5nKHRleHRvKTpcbiAgcmVzcG9uc2UgPSBnZW1pbmkuZ2VuZXJhdGVfZW1iZWRkaW5ncyhtb2RlbD1cImdlbWluaS1wcm9cIiwgdGV4dHM9W3RleHRvXSlcbiAgcmV0dXJuIHJlc3BvbnNlLmVtYmVkZGluZ3NbMF1cblxuIyBDcmlhciBjb2xlw6fDo28gbm8gQ2hyb21hXG5jb2xlY2FvID0gY2xpZW50LmNyZWF0ZV9jb2xsZWN0aW9uKG5hbWU9XCJkb2N1bWVudG9zXCIsIGVtYmVkZGluZ19mdW5jdGlvbj1lbWJlZGRpbmdfZnVuY3Rpb25zLkdlbWluaSgpKVxuXG4jIEluZGV4YXIgZG9jdW1lbnRvc1xuZG9jdW1lbnRvcyA9IFtcbiAgICB7XCJpZFwiOiAxLCBcInRleHRvXCI6IFwiRXN0ZSDDqSB1bSBleGVtcGxvIGRlIGRvY3VtZW50byBzb2JyZSBnYXRvcy5cIiwgXCJtZXRhZGFkb3NcIjoge1widGl0dWxvXCI6IFwiR2F0b3NcIn19LFxuICAgIHtcImlkXCI6IDIsIFwidGV4dG9cIjogXCJFc3RlIMOpIG91dHJvIGV4ZW1wbG8gZGUgZG9jdW1lbnRvIHNvYnJlIGPDo2VzLlwiLCBcIm1ldGFkYWRvc1wiOiB7XCJ0aXR1bG9cIjogXCJDw6Nlc1wifX1cbl1cbmZvciBkb2MgaW4gZG9jdW1lbnRvczpcbiAgICBlbWJlZGRpbmcgPSBnZXJhcl9lbWJlZGRpbmcoZG9jW1widGV4dG9cIl0pXG4gICAgY29sZWNhby5hZGQoXG4gICAgICAgIGVtYmVkZGluZ3M9W2VtYmVkZGluZ10sXG4gICAgICAgIG1ldGFkYXRhcz1bZG9jW1wibWV0YWRhZG9zXCJdXSxcbiAgICAgICAgaWRzPVtzdHIoZG9jW1wiaWRcIl0pXVxuICAgIClcblxuIyBDb25zdWx0YXIgZG9jdW1lbnRvc1xuY29uc3VsdGEgPSBcImFuaW1haXMgZGUgZXN0aW1hw6fDo29cIlxuZW1iZWRkaW5nX2NvbnN1bHRhID0gZ2VyYXJfZW1iZWRkaW5nKGNvbnN1bHRhKVxucmVzdWx0YWRvcyA9IGNvbGVjYW8ucXVlcnkoXG4gICAgcXVlcnlfZW1iZWRkaW5ncz1bZW1iZWRkaW5nX2NvbnN1bHRhXSxcbiAgICBuX3Jlc3VsdHM9MixcbiAgICBpbmNsdWRlPVtcIm1ldGFkYXRhc1wiLCBcImRpc3RhbmNlc1wiXVxuKVxuXG4jIE1vc3RyYXIgcmVzdWx0YWRvc1xuZm9yIHJlc3VsdGFkbyBpbiByZXN1bHRhZG9zW1wibWV0YWRhdGFzXCJdWzBdOlxuICAgIHByaW50KGZcIlTDrXR1bG86IHtyZXN1bHRhZG9bJ3RpdHVsbyddfSwgRGlzdMOibmNpYToge3Jlc3VsdGFkb3NbJ2Rpc3RhbmNlcyddWzBdfVwiKVxuYGBgXG5cbioqQ29uc2lkZXJhw6fDtWVzOioqXG5cbiogKipFc2NhbGFiaWxpZGFkZToqKiBQYXJhIGdyYW5kZXMgdm9sdW1lcyBkZSBkYWRvcywgb3RpbWl6YXIgbyBiYW5jbyBkZSBkYWRvcyBlIGEgYnVzY2Egc2Vyw6EgY3J1Y2lhbC5cbiogKipNw6l0cmljYXMgZGUgc2ltaWxhcmlkYWRlOioqIEV4cGxvcmUgZGlmZXJlbnRlcyBtw6l0cmljYXMgZGUgc2ltaWxhcmlkYWRlIHBhcmEgZW5jb250cmFyIGEgcXVlIG1lbGhvciBzZSBhanVzdGEgYW9zIHNldXMgZGFkb3MuXG4qICoqQXR1YWxpemHDp8OjbyBkbyDDrW5kaWNlOioqIEltcGxlbWVudGUgdW0gcHJvY2Vzc28gcGFyYSBhdHVhbGl6YXIgbyDDrW5kaWNlIGNvbSBub3ZvcyBkb2N1bWVudG9zLlxuKiAqKkludGVyZmFjZSBkbyB1c3XDoXJpbzoqKiBDcmllIHVtYSBpbnRlcmZhY2UgaW50dWl0aXZhIHBhcmEgbyB1c3XDoXJpbyByZWFsaXphciBidXNjYXMgZSB2aXN1YWxpemFyIG9zIHJlc3VsdGFkb3MuXG5cbkVzdGUgc2lzdGVtYSBkZW1vbnN0cmEgdW0gZXhlbXBsbyBiw6FzaWNvIGRlIGJ1c2NhIGNvbSBlbWJlZGRpbmdzIGUgYSBHZW1pbmkgQVBJLiBWb2PDqiBwb2RlIGFkYXB0w6EtbG8gZSBleHBhbmRpLWxvIGRlIGFjb3JkbyBjb20gc3VhcyBuZWNlc3NpZGFkZXMgZSByZWN1cnNvcyBlc3BlY8OtZmljb3MuIn1d' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjowLCJtYXhfb3V0cHV0X3Rva2VucyI6ODE5Miwic3RvcF9zZXF1ZW5jZXMiOltdfQ==' # @param {isTemplate: true}
safety_settings_b64 = 'W3siY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX0hBUkFTU01FTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfSEFURV9TUEVFQ0giLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfU0VYVUFMTFlfRVhQTElDSVQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfREFOR0VST1VTX0NPTlRFTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn1d' # @param {isTemplate: true}
user_input_b64 = '' # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))
generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))
user_input = base64.b64decode(user_input_b64).decode()
stream = False

In [ ]:
contents

[{'role': 'user', 'parts': [{'text': 'hello'}]},
 {'role': 'model', 'parts': [{'text': 'Hello! How can I assist you today?'}]}]

In [ ]:
generation_config

{}

In [ ]:
safety_settings

{}

In [ ]:
user_input

'How does electricity work?'

### Call the API

In [ ]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

chat = gemini.start_chat(history=contents)

response = chat.send_message(
    user_input,
    stream=stream)

In [ ]:
display(Markdown(response.text))

Electricity is the flow of electric charge. It is a fundamental part of nature and is responsible for a wide range of phenomena, from lightning and static electricity to the operation of electronic devices.

**Basic Principles of Electricity:**

1. **Electric Charge:** Matter is made up of atoms, which have a positively charged nucleus surrounded by negatively charged electrons. The movement of these charged particles creates electric currents.

2. **Electric Field:** Electric charges create an electric field around them. The strength and direction of this field depend on the amount and type of charge.

3. **Electric Potential:** Electric potential is the amount of electrical energy stored in a region of space. It is measured in volts (V).

4. **Electric Current:** Electric current is the flow of electric charge through a conductor. It is measured in amperes (A).

**How Electricity Works:**

1. **Generation of Electricity:** Electricity can be generated in various ways, including:
   - **Mechanical Energy:** Generators convert mechanical energy into electrical energy by spinning a conductor in a magnetic field. This is the principle behind power plants that use turbines driven by steam, water, or wind.
   - **Chemical Energy:** Batteries generate electricity through chemical reactions.
   - **Solar Energy:** Solar cells convert sunlight directly into electricity using the photovoltaic effect.

2. **Transmission of Electricity:** Electricity is transmitted over long distances through power lines. These lines are made of conductive materials, such as copper or aluminum, which allow electrons to flow easily.

3. **Distribution of Electricity:** Once electricity reaches populated areas, it is distributed to homes, businesses, and other consumers through a network of local power lines and transformers.

4. **Utilization of Electricity:** Electricity is used to power various devices and appliances by converting electrical energy into other forms of energy, such as light, heat, or motion.

The flow of electricity in a circuit is driven by the difference in electric potential between two points. When a complete circuit is formed, electrons can flow from a region of higher potential to a region of lower potential, creating an electric current.

Electricity is a versatile form of energy that has revolutionized modern society. It is used in a wide range of applications, from lighting and heating to powering computers and transportation systems.

In [ ]:
response.prompt_feedback

safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}

In [ ]:
response.candidates

[index: 0
content {
  parts {
    text: "Electricity is the flow of electric charge. It is a fundamental part of nature and is responsible for a wide range of phenomena, from lightning and static electricity to the operation of electronic devices.\n\n**Basic Principles of Electricity:**\n\n1. **Electric Charge:** Matter is made up of atoms, which have a positively charged nucleus surrounded by negatively charged electrons. The movement of these charged particles creates electric currents.\n\n2. **Electric Field:** Electric charges create an electric field around them. The strength and direction of this field depend on the amount and type of charge.\n\n3. **Electric Potential:** Electric potential is the amount of electrical energy stored in a region of space. It is measured in volts (V).\n\n4. **Electric Current:** Electric current is the flow of electric charge through a conductor. It is measured in amperes (A).\n\n**How Electricity Works:**\n\n1. **Generation of Electricity:** Electr